### Pip install

In [ ]:
pip install torchtext

In [ ]:
pip install torch torchvision

In [ ]:
pip install nltk

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

### Import

In [1]:
# Import list

import numpy as np
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
# from sklearn.preprocessing import LabelBinarizer
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Import ReduceLROnPlateau
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Read Data 

In [17]:
# Different Editors paths
PATH_A = "data\sherlock-holm.es_stories_plain-text_advs.txt"
PATH_E = ""
PATH_G = "0. Projects/3/Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"
PATH_J = ""
PATH_M = "Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

PATHS = [PATH_A, PATH_E, PATH_G, PATH_J, PATH_M]

text = ""

for path in PATHS:
    try:
        # Read the text file
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read()
            print(path)
    except:
        continue
    else:
        break



print(text[:200])

data\sherlock-holm.es_stories_plain-text_advs.txt




                        THE ADVENTURES OF SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Scandal in Bohem


## Data Preprocess

### Divide the set

In [18]:
# Divide with regular expressions
DIVIDERS_ORIGINAL = "\n"
DIVIDERS_ALL = "[,.!?:;\"]|\n\n|--| and | that | which "
DIVIDERS_MIN = "[.!]|\n\n"
DIVIDERS_BAL = "[,.!?]|\n\n|--"
CLEAR_COVER = False

text_try = text.lower()

if CLEAR_COVER:
    # Delete cover of book and extra information
    text_try = text[980:-550]
    

# Split following the dividers given
text_try = re.split(DIVIDERS_ORIGINAL, text_try)

# Delete all the new line comments 
text_try = [el.replace('\n', '') for el in text_try]

text_try[:10]

['',
 '',
 '',
 '',
 '                        the adventures of sherlock holmes',
 '',
 '                               arthur conan doyle',
 '',
 '',
 '']

### Tokenization

In [19]:
# Create Tokenizer object in python
CLEAR_COVER
tokens = word_tokenize(text)
vocabulary = set(tokens)
total_words = len(vocabulary) + 1

word_to_idx = {word:idx for idx, word in enumerate(vocabulary)}

print(f"total_words: {total_words}")
print(f"Índice de palabras: {word_to_idx}")

total_words: 8965
Índice de palabras: {'Therein': 0, 'swordsman': 1, 'morning': 2, 'self-control': 3, 'pooh': 4, 'coach-house': 5, 'shift': 6, 'height': 7, 'experience': 8, 'accomplish': 9, 'true': 10, 'minister': 11, 'Jones': 12, 'assist': 13, 'hers': 14, 'solution': 15, 'turf': 16, 'occupations': 17, 'match': 18, "'band": 19, 'fluffy': 20, 'hand-mirror': 21, 'horror-stricken': 22, 'explains': 23, 'imagination': 24, 'blow': 25, 'hill': 26, 'gentlemen': 27, 'lodgings': 28, 'aisle': 29, 'simple-minded': 30, 'keep': 31, 'sake': 32, 'consuming': 33, 'Prince': 34, 'rattle': 35, 'begun': 36, 'danseuse': 37, 'Calcutta': 38, 'unknown': 39, 'Contralto': 40, 'kind-hearted': 41, 'compressed': 42, 'logical': 43, 'camera': 44, 'dusty': 45, 'hawk-like': 46, 'begged': 47, 'Wedlock': 48, 'appearance': 49, 'bride': 50, 'Theological': 51, 'poison': 52, 'candle': 53, 'crowded': 54, 'grinning': 55, 'diamond-shaped': 56, 'menaced': 57, 'forty': 58, 'victory': 59, 'gate': 60, 'ship': 61, 'lank': 62, 'purse

In [20]:
# Create input-output sequences
input_sequences = []
for line in text_try:
    line_list = line.rstrip(",.;:").split(' ')

    # Tokenize each sentence
    token_list = []
    for char in line_list:
        if char in word_to_idx.keys():
            token_list.append(word_to_idx[char])

    # Divide the different sentences in n-grams
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences


[[4643, 4281],
 [4643, 4281, 4740],
 [4860, 4740],
 [4860, 4740, 3258],
 [4566, 1761],
 [4566, 1761, 5565],
 [4643, 2097],
 [4643, 2097, 8308],
 [4566, 4146],
 [4566, 4146, 4740],
 [4566, 4146, 4740, 7797],
 [4643, 3297],
 [4643, 653],
 [4643, 653, 8844],
 [4643, 653, 8844, 1001],
 [4643, 690],
 [4643, 690, 2663],
 [4643, 690, 2663, 4643],
 [4643, 690, 2663, 4643, 1202],
 [4643, 690, 2663, 4643, 1202, 5896],
 [4643, 6727],
 [4643, 6727, 4740],
 [4643, 6727, 4740, 4643],
 [4643, 6727, 4740, 4643, 4411],
 [4643, 6727, 4740, 4643, 4411, 8834],
 [4643, 6727],
 [4643, 6727, 4740],
 [4643, 6727, 4740, 4643],
 [4643, 6727, 4740, 4643, 3535],
 [4643, 6727, 4740, 4643, 3535, 6325],
 [4643, 6727],
 [4643, 6727, 4740],
 [4643, 6727, 4740, 4643],
 [4643, 6727, 4740, 4643, 7238],
 [4643, 6727],
 [4643, 6727, 4740],
 [4643, 6727, 4740, 4643],
 [4643, 6727, 4740, 4643, 5659],
 [4643, 6727, 4740, 4643, 5659, 6414],
 [4643, 6727],
 [4643, 6727, 4740],
 [4643, 6727, 4740, 4643],
 [4643, 6727, 4740, 4643

In [21]:
import torch
torch.cuda.is_available()



True

### Padding

In [22]:
# Get the max value to add padding to other entries

average = 0
for seq in input_sequences:
    average += len(seq) 

max_sequence_len, value = max([(len(seq), seq) for seq in input_sequences])
input_seq_pad = np.array([np.pad(seq, (max_sequence_len - len(seq), 0), mode='constant') for seq in input_sequences])

print (f"average = {average / len(input_sequences)}")
print (f"Max seq length = {max_sequence_len}")
input_seq_pad

average = 6.213641593749164
Max seq length = 16


array([[   0,    0,    0, ...,    0, 4643, 4281],
       [   0,    0,    0, ..., 4643, 4281, 4740],
       [   0,    0,    0, ...,    0, 4860, 4740],
       ...,
       [   0,    0,    0, ..., 4988,  691, 4643],
       [   0,    0,    0, ...,  691, 4643, 6536],
       [   0,    0,    0, ..., 4643, 6536, 5563]])

## Model Train

### X and Y separation

In [26]:
# Split the sequences into input (X) and output (y)
X = input_seq_pad[:, :-1]
y = input_seq_pad[:, -1]

y_tensor = torch.tensor(y, dtype=torch.int64)
y = F.one_hot(y_tensor, num_classes=total_words)


In [27]:
# Create a custom Dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = TextDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_23620\2181850307.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.long)


## MODELS

### Multiple LSTM layers

In [51]:
# Define the model
class NextWordPredictor(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, dropout=0.3):
    super(NextWordPredictor, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.dropout_embed = nn.Dropout(dropout)  # Add dropout after embedding
    self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2, dropout=dropout, bidirectional=True, batch_first=True)  # Use Bidirectional LSTM with multiple layers
    self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Adjust output size for Bidirectional LSTM

  def forward(self, sequences):
    embedded = self.embedding(sequences)
    embedded = self.dropout_embed(embedded)
    lstm_out, _ = self.lstm(embedded)
    last_hidden = lstm_out[:, -1, :]  # Select last hidden state from the sequence
    logits = self.fc(last_hidden)
    return logits



### Simple LSTM

In [66]:
# Train the model
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim,dropout=0.2):
        super(NextWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.dropout_embed = nn.Dropout(dropout)  # Add dropout after embedding
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout_embed(embedded)
        x, _ = self.lstm(embedded)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [67]:
model = NextWordPredictor(vocab_size = total_words, 
embed_dim = 450, hidden_dim = 600, output_dim = total_words).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



## Training

In [68]:
# Coda setting
import torch
print(f"Is coda avialable: {torch.cuda.is_available()}")

a=torch.FloatTensor([1.0,2.0]).cuda()
a.device
torch.manual_seed(32)


next(model.parameters()).is_cuda
gpumodel = model.cuda()

Is coda avialable: True


In [69]:
SAVE_MODEL = False  #If set to True, saves a torch_model.pt 

epochs = 200
patience = 10  # Number of epochs to wait for improvement


current_patience = patience
best_loss = float('inf')  # Initialize best loss to a very high value
better_model = model
for epoch in range(epochs):
  # Training loop
  for i, (X_batch, y_batch) in enumerate(dataloader):
    X_batch, y_batch = X_batch.cuda(), y_batch.cuda()  # Mover datos a la GPU
    outputs = model(X_batch)
    loss = criterion(outputs, y_batch.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Early stopping
  if loss.item() < best_loss:  # Compare current training loss with best loss
    best_loss = loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()} (Improved)')
    better_model = model
    current_patience = patience  # Restart patience
  else:
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
    current_patience -= 1  # Decrement patience counter on no improvement

  # Stop training if patience is 0
  if current_patience == 0:
    print('Early stopping triggered!')
    break

#Saving model to .pt
if SAVE_MODEL == True:
  torch.save(better_model.state_dict(), 'best_model_simple_LSTM.pt')

Epoch 1/200, Loss: 5.193134307861328 (Improved)
Epoch 2/200, Loss: 4.992818355560303 (Improved)
Epoch 3/200, Loss: 4.816873073577881 (Improved)
Epoch 4/200, Loss: 3.603431224822998 (Improved)
Epoch 5/200, Loss: 3.4084668159484863 (Improved)
Epoch 6/200, Loss: 2.6486408710479736 (Improved)
Epoch 7/200, Loss: 2.2045345306396484 (Improved)
Epoch 8/200, Loss: 1.762470006942749 (Improved)
Epoch 9/200, Loss: 1.7109382152557373 (Improved)
Epoch 10/200, Loss: 1.688164472579956 (Improved)
Epoch 11/200, Loss: 1.5012282133102417 (Improved)
Epoch 12/200, Loss: 1.267298936843872 (Improved)
Epoch 13/200, Loss: 0.8364430665969849 (Improved)
Epoch 14/200, Loss: 0.9606257677078247
Epoch 15/200, Loss: 1.3490160703659058
Epoch 16/200, Loss: 1.0428862571716309
Epoch 17/200, Loss: 0.6408182978630066 (Improved)
Epoch 18/200, Loss: 1.005302906036377
Epoch 19/200, Loss: 0.7519671320915222
Epoch 20/200, Loss: 0.6617203950881958
Epoch 21/200, Loss: 0.7232803106307983
Epoch 22/200, Loss: 0.9269039034843445
Epoch

## Prediction

In [71]:
# Initial text to predict
seed_text = "I am"
next_words = 15

# Index to word
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

device = torch.device('cpu')
better_model.to(device)  # Asegurarse de que el modelo esté en el dispositivo correcto
# Generate the n next words
better_model.eval()  # Set the model to evaluation
for _ in range(next_words):
    tokens = word_tokenize(seed_text)
    token_list = [word_to_idx[word] for word in tokens if word in word_to_idx]
    token_list = np.pad(token_list, (max_sequence_len - len(token_list), 0), mode='constant')
    token_list = torch.tensor(token_list[-max_sequence_len:], dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        predicted = better_model(token_list).argmax(dim=1).item()

    output_word = idx_to_word[predicted]
    seed_text += " " + output_word


print(seed_text)

I am much is the person in not arresting a said he thought that the man who
